# Analyzing the Free Mobile App Market
--DESCRIPTION 1-2 paragraphs explaining what the project is about and its goals--
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

### Loading the Data and First Look
Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of the data instead. To avoid spending resources on collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our goals:

- A [data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play — the data was collected in August 2018
- A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store — the data was collected in July 2017
We'll start by opening and exploring these two data sets. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [2]:
google_store_df = pd.read_csv('googleplaystore.csv')
apple_store_df = pd.read_csv('AppleStore.csv')

Let's look at the first few rows of the Google Play store data:

In [3]:
print('''Google Play store 
          # of apps:    {} 
          # of columns: {}
      '''.format(*google_store_df.shape))
print(google_store_df.head(3))

Google Play store 
          # of apps:    10841 
          # of columns: 13
      
                                                 App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   

  Reviews  Size    Installs  Type Price Content Rating  \
0     159   19M     10,000+  Free     0       Everyone   
1     967   14M    500,000+  Free     0       Everyone   
2   87510  8.7M  5,000,000+  Free     0       Everyone   

                      Genres      Last Updated Current Ver   Android Ver  
0               Art & Design   January 7, 2018       1.0.0  4.0.3 and up  
1  Art & Design;Pretend Play  January 15, 2018       2.0.0  4.0.3 and up  
2               Art & Design    August 1, 2018       1.2.4  4.0.3 and up  


We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

Now let's look at the iOS App store data set.

In [4]:
print('''Apple store 
          # of apps:    {} 
          # of columns: {}
      '''.format(*apple_store_df.shape))
print(apple_store_df.head(3))

Apple store 
          # of apps:    7197 
          # of columns: 17
      
   Unnamed: 0         id                                       track_name  \
0           1  281656475                                  PAC-MAN Premium   
1           2  281796108                        Evernote - stay organized   
2           3  281940292  WeatherBug - Local Weather, Radar, Maps, Alerts   

   size_bytes currency  price  rating_count_tot  rating_count_ver  \
0   100788224      USD   3.99             21292                26   
1   158578688      USD   0.00            161065                26   
2   100524032      USD   0.00            188583              2822   

   user_rating  user_rating_ver    ver cont_rating   prime_genre  \
0          4.0              4.5  6.3.5          4+         Games   
1          4.0              3.5  8.2.2          4+  Productivity   
2          3.5              4.5  5.0.0          4+       Weather   

   sup_devices.num  ipadSc_urls.num  lang.num  vpp_lic  
0      

The Apple Store data set has 7197 apps and the columns that look interesting are 'track_name', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. A more complete description of each column can be found in the [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

### Data Cleaning
Next let's look through the data for possible erroneous input and pare it down to our needs.

#### Google Play
The [Kaggle Discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/81616) mention a few mistakes found in the Google Play data:
1. Several apps appear more than once in the data: 

In [5]:
print('Number of duplicate apps:', len(google_store_df) - google_store_df['App'].nunique())
google_store_df[(google_store_df['App'] == 'Subway Surfers') | (google_store_df['App'] == 'Facebook')]

Number of duplicate apps: 1181


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1654,Subway Surfers,GAME,4.5,27722264,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up
1700,Subway Surfers,GAME,4.5,27723193,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up
1750,Subway Surfers,GAME,4.5,27724094,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up
1872,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up
1917,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up
2544,Facebook,SOCIAL,4.1,78158306,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"August 3, 2018",Varies with device,Varies with device
3896,Subway Surfers,GAME,4.5,27711703,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up
3943,Facebook,SOCIAL,4.1,78128208,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"August 3, 2018",Varies with device,Varies with device


Let's find and remove these duplicate entries, but first notice that all of the columns appear to be the same except for the 'Reviews' column, so which do we keep? Rather than throw away the rows randomly, let's get rid of all but the one with the highest number of reviews as it likely has the most up to date information on the app.

In [6]:
google_store_df = google_store_df.sort_values('Reviews').drop_duplicates(subset='App', keep='last')
print('Apps remaining in data: ', len(google_store_df))
google_store_df[(google_store_df['App'] == 'Subway Surfers') | (google_store_df['App'] == 'Facebook')]

Apps remaining in data:  9660


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1917,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up
2544,Facebook,SOCIAL,4.1,78158306,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"August 3, 2018",Varies with device,Varies with device


2. Row 10472 has displaced columns where 'Category' is '1.9' and the 'Rating' is '19' even though the rating is supposed to be in a range 0-5. Let's remove that row as well.

In [7]:
google_store_df.loc[10472]

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              1.9
Rating                                                 19
Reviews                                              3.0M
Size                                               1,000+
Installs                                             Free
Type                                                    0
Price                                            Everyone
Content Rating                                        NaN
Genres                                  February 11, 2018
Last Updated                                       1.0.19
Current Ver                                    4.0 and up
Android Ver                                           NaN
Name: 10472, dtype: object

In [8]:
google_store_df.drop(10472, inplace=True)
print('Apps remaining in data:', len(google_store_df))

Apps remaining in data: 9659


Since we are looking at the free app market for a primarily English-speaking audience, let's remove the non-free apps and non-English apps from the data set.